# Redes Neuronales Convolucionales (convets o CNN)

Se trata de un tipo de modelo casi universal en la aplicación de visión por ordenador.

In [10]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [11]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Los _convet_ toma como entrada tensores con la forma (altura_imagen, anchura_imagen, canales_imagen). En este caso, configuramos las _convet_ para que procese la entrada de tamaño (28, 28, 1), que es el formato de las imagenes MNIST. input_shape=(28, 28, 1).

Las capas Conv2D aplican un conjunto de filtros (también conocidos como kernels) a una entrada bidimensional (por ejemplo, una imagen) para crear un conjunto de mapas de características. 

In [3]:
# Resumen de la arquitectura de la convet
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
Total params: 55,744
Trainable params: 55,744
Non-traina

La salida de cada capa _Conv2D_ y _MaxPooling2D_ es un tensor 3D con la forma (altura, anchura, canales).

La dimensión de anchura y altura tienden a reducirse a medida que profundizamos en la red.

El número de canales está controlado por el primer argumento pasado en las capas _Conv2D_ (32 O 64).

In [ ]:
# NO CORRER
model.add(Conv2D(filters=32, 
                 kernel_size=(3, 3), 
                 strides=(1, 1), 
                 padding='valid', 
                 activation='relu', 
                 input_shape=(64, 64, 3)))

- filters=32: Especifica que habrá 32 filtros/kernels diferentes en la capa.
- kernel_size=(3, 3): Define el tamaño de los filtros como 3x3.
- strides=(1, 1): Establece el stride a 1 en ambas dimensiones.
- padding='valid': Indica que no se utilizará padding.
- activation='relu': Usa la función de activación ReLU.
- input_shape=(64, 64, 3): Define la forma de la entrada, en este caso, imágenes de 64x64 con 3 canales (RGB).

El siguiente paso es introducir el último tensor con entrada (3, 3, 64) en una red clasificadora densamente conectada, pilas _Dense_. Estas redes procesan vectores 1D, mentras que la salida actual es un Tensor 3D. Entonces, tenemos que aplanar la salida de 3D a 1D, y luego aplicar capas _Dense_.

In [12]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 576)              

Como se observa, la salida (3, 3, 64) se ha aplanado para convertirse en vectores con forma (576,), antes de pasar por dos capas _Dense_.

In [14]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 20s 20ms/step - loss: 0.1764 - accuracy: 0.9449
Epoch 2/5
938/938 [==============================] - 21s 22ms/step - loss: 0.0467 - accuracy: 0.9859
Epoch 3/5
938/938 [==============================] - 20s 22ms/step - loss: 0.0326 - accuracy: 0.9900
Epoch 4/5
938/938 [==============================] - 20s 21ms/step - loss: 0.0253 - accuracy: 0.9920
Epoch 5/5
938/938 [==============================] - 20s 21ms/step - loss: 0.0196 - accuracy: 0.9939


In [15]:
# Evaluación del modelo
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

313/313 [==============================] - 1s 4ms/step - loss: 0.0254 - accuracy: 0.9923


0.9922999739646912